In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = 'output_data/cities.csv'

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [13]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        
#OpenWeather Key
api_key = weather_api
units = "imperial"

query_url = f"http://api.openweathermap.org/data/2.5/weather?appid={api_key}&units={units}&q="

# Print the city count to confirm sufficient count
len(cities)

595

In [14]:
#Empty lists for my variables
cloudiness = []
date = []
humidity = []
lat = []
lng = []
max_temp = []
wind_speed = []
country = []
city_column = []

count = 0
sets = 1

In [15]:
#cities


In [16]:
#Run the API call for each city
print('Beginning Data Retrieval')
print('-----------------------------')

for city in cities:
    weather_json = requests.get(query_url + city.replace(" ","&")).json()
    if count < 50:
            count += 1
            city_column.append(city)
    else:
            sets += 1
            count = 0
            city_column.append(city)
    print(f"Processing Record {count} of Set {sets} | {city}")
    try:
        cloudiness.append(weather_json['clouds']['all'])
        country.append(weather_json['sys']['country'])
        date.append(weather_json['dt'])
        humidity.append(weather_json['main']['humidity'])
        max_temp.append(weather_json['main']['temp_max'])
        lat.append(weather_json['coord']['lat'])
        lng.append(weather_json['coord']['lon'])
        wind_speed.append(weather_json['wind']['speed'])
        city_column.append(city)
    except:
        print('City not found. Skipping...')
    pass
print('------------------------------')
print('Data Retrieval Complete')
print('------------------------------')

Beginning Data Retrieval
-----------------------------
Processing Record 1 of Set 1 | norman wells
Processing Record 2 of Set 1 | codrington
Processing Record 3 of Set 1 | atuona
Processing Record 4 of Set 1 | hermanus
Processing Record 5 of Set 1 | torbay
Processing Record 6 of Set 1 | punta arenas
Processing Record 7 of Set 1 | san jose
Processing Record 8 of Set 1 | rikitea
Processing Record 9 of Set 1 | ancud
Processing Record 10 of Set 1 | saldanha
Processing Record 11 of Set 1 | hobart
Processing Record 12 of Set 1 | santander
Processing Record 13 of Set 1 | bure
Processing Record 14 of Set 1 | jamestown
Processing Record 15 of Set 1 | dikson
Processing Record 16 of Set 1 | provideniya
Processing Record 17 of Set 1 | busselton
Processing Record 18 of Set 1 | caravelas
Processing Record 19 of Set 1 | west wendover
Processing Record 20 of Set 1 | bathsheba
Processing Record 21 of Set 1 | kapaa
Processing Record 22 of Set 1 | samarai
Processing Record 23 of Set 1 | mehamn
Processing

Processing Record 36 of Set 4 | dingle
Processing Record 37 of Set 4 | stratonion
Processing Record 38 of Set 4 | hofn
Processing Record 39 of Set 4 | batagay-alyta
Processing Record 40 of Set 4 | borova
Processing Record 41 of Set 4 | kimbe
Processing Record 42 of Set 4 | maple creek
Processing Record 43 of Set 4 | ribeira grande
Processing Record 44 of Set 4 | ust-kuyga
Processing Record 45 of Set 4 | carutapera
Processing Record 46 of Set 4 | otane
Processing Record 47 of Set 4 | samusu
City not found. Skipping...
Processing Record 48 of Set 4 | mys shmidta
Processing Record 49 of Set 4 | kyabe
Processing Record 50 of Set 4 | athabasca
Processing Record 0 of Set 5 | kruisfontein
Processing Record 1 of Set 5 | solsvik
City not found. Skipping...
Processing Record 2 of Set 5 | datong
Processing Record 3 of Set 5 | burgeo
Processing Record 4 of Set 5 | viedma
Processing Record 5 of Set 5 | kosh-agach
Processing Record 6 of Set 5 | alofi
Processing Record 7 of Set 5 | narsaq
Processing 

Processing Record 19 of Set 8 | manono
Processing Record 20 of Set 8 | khilok
Processing Record 21 of Set 8 | bonavista
Processing Record 22 of Set 8 | tianpeng
Processing Record 23 of Set 8 | tupik
Processing Record 24 of Set 8 | burlatskoye
Processing Record 25 of Set 8 | leh
Processing Record 26 of Set 8 | ivanjica
Processing Record 27 of Set 8 | tuatapere
Processing Record 28 of Set 8 | kitimat
Processing Record 29 of Set 8 | touros
Processing Record 30 of Set 8 | sainte-thecle
Processing Record 31 of Set 8 | evensk
Processing Record 32 of Set 8 | vanimo
Processing Record 33 of Set 8 | kutum
Processing Record 34 of Set 8 | dhanwar
Processing Record 35 of Set 8 | palana
Processing Record 36 of Set 8 | redmond
Processing Record 37 of Set 8 | strezhevoy
Processing Record 38 of Set 8 | itaituba
Processing Record 39 of Set 8 | beira
Processing Record 40 of Set 8 | guntakal
Processing Record 41 of Set 8 | jasper
Processing Record 42 of Set 8 | asilah
Processing Record 43 of Set 8 | tilic

Processing Record 2 of Set 12 | kankon
Processing Record 3 of Set 12 | mineros
Processing Record 4 of Set 12 | zhezkazgan
Processing Record 5 of Set 12 | emerald
Processing Record 6 of Set 12 | bayburt
Processing Record 7 of Set 12 | sao felix do xingu
Processing Record 8 of Set 12 | puerto del rosario
Processing Record 9 of Set 12 | solnechnyy
Processing Record 10 of Set 12 | milingimbi
City not found. Skipping...
Processing Record 11 of Set 12 | chaman
Processing Record 12 of Set 12 | dixon
Processing Record 13 of Set 12 | luebo
Processing Record 14 of Set 12 | santona
Processing Record 15 of Set 12 | boa vista
Processing Record 16 of Set 12 | jawhar
Processing Record 17 of Set 12 | vostok
Processing Record 18 of Set 12 | umzimvubu
City not found. Skipping...
Processing Record 19 of Set 12 | vilyuysk
Processing Record 20 of Set 12 | kamyshla
Processing Record 21 of Set 12 | rudbar
Processing Record 22 of Set 12 | mandaguacu
Processing Record 23 of Set 12 | ullapool
Processing Record 

In [10]:
#create new dataframe using new columns
weather_dict = {
    "City": city_column,
    "Cloudiness": cloudiness,
    "Country": country,
    "Date": date,
    "Humidity": humidity,
    "Lat": lat,
    "Lng": lng,
    "Max Temp": max_temp,
    "Wind Speed": wind_speed}

In [11]:
weather_df = pd.DataFrame(weather_dict)
weather_df.head()

ValueError: arrays must all be same length

In [ ]:
#weather_df.to_csv(output_data_file)

In [ ]:
plt.scatter(weather_df["Lat"],weather_df["Max Temp"],edgecolors="black",facecolors="blue")
plt.title("City Latitude vs. Max Temperature (07/23/2020)")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.xlim(-60,80)
plt.ylim(-30,100)
plt.grid (b=True,which="major",axis="both",linestyle="-",color="lightgrey")
#plt.savefig("Figures/fig1.png")
plt.show()

In [ ]:
plt.scatter(weather_df["Lat"],weather_df["Humidity"],edgecolors="black",facecolors="blue")
plt.title("City Latitude vs. Humidity (07/23/2020)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.ylim(15,105)
plt.grid (b=True,which="major",axis="both",linestyle="-",color="lightgrey")
#plt.savefig("Figures/fig2.png")
plt.show()

In [ ]:
plt.scatter(weather_df["Lat"],weather_df["Cloudiness"],edgecolors="black",facecolors="blue")
plt.title("City Latitude vs. Cloudiness (07/23/2020)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid (b=True,which="major",axis="both",linestyle="-",color="lightgrey")
#plt.savefig("Figures/fig3.png")
plt.show()

In [ ]:
plt.scatter(weather_df["Lat"],weather_df["Wind Speed"],edgecolors="black",facecolors="blue")
plt.title("City Latitude vs. Wind Speed (07/23/2020)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.ylim(-2,34)
plt.grid (b=True,which="major",axis="both",linestyle="-",color="lightgrey")
#plt.savefig("Figures/fig4.png")
plt.show()